# Register Custom Monitor in OpenScale
This notebook 
1. registers the custom metric provider as an integrated system in openscale
2. creates the corresponding monitor definition
3. updates monitor metadata

In [1]:
from ibm_watson_openscale import *
from ibm_watson_openscale.base_classes.watson_open_scale_v2 import *
import urllib3
urllib3.disable_warnings()

from datetime import datetime, timezone, timedelta
import uuid

import os
import wos_sdk_utils as wos_util

In [2]:
scoring_url = 'https://internal-nginx-svc:12443/ml/v4/deployments/af5ea879-60b8-4f9e-a276-893e1f986a34/predictions?version=2022-05-03'
openscale_integrated_system_name = "dummy metrics example dima"
openscale_monitor_name = 'dummy monitor example dima'

credentials = {'username': 'kharlad',
               'api_key': os.environ['USER_ACCESS_TOKEN']}

## 1. Register Scoring Endpoint in OpenScale

In [3]:
wos_client = wos_util.get_client()
wos_client.version

'3.0.28'

In [4]:
# Delete existing custom metrics provider integrated systems if present
wos_util.integrated_system_delete(openscale_integrated_system_name,wos_client)

Deleted integrated system dummy metrics example dima


In [5]:
custom_metrics_integrated_system = IntegratedSystems(wos_client).add(
    name=openscale_integrated_system_name,
    description=openscale_integrated_system_name,
    type="custom_metrics_provider",
    credentials= {"auth_type":"bearer",
                  "token_info": {
                      "url": "{}/icp4d-api/v1/authorize".format(os.environ['RUNTIME_ENV_APSX_URL']),
                      "headers": {"Content-Type": "application/json",
                                  "Accept": "application/json"},
                      "payload": {'username':credentials['username'],
                                   'api_key':credentials['api_key']},
                      "method": "post"}
                 },
    connection={"display_name": openscale_integrated_system_name,
                "endpoint": scoring_url
    }).result

integrated_system_id = custom_metrics_integrated_system.metadata.id
print(integrated_system_id)

17806f17-6014-438e-8321-945e17a17ddc


## 2. Setup Custom Monitor Definition

In [6]:
openscale_monitor_defaults = {'sensitivity': {'threshold':[0.8,'lower']},
                              'specificity': {'threshold':[0.9,'lower']}
                             }

In [7]:
monitor_id = wos_util.monitor_definition_create(openscale_monitor_name,
                                                openscale_monitor_defaults,
                                                wos_client,overwrite=True)
print(monitor_id)

Deleted existing monitor definition dummy monitor example dima (dummy_monitor_example_dima)



 Waiting for end of adding monitor definition dummy_monitor_example_dima 




finished

-------------------------------------------------
 Successfully finished adding monitor definition 
-------------------------------------------------


dummy_monitor_example_dima


In [8]:
wos_client.monitor_definitions.show()

dummy_monitor_example_dima,dummy monitor example dima,"['sensitivity', 'specificity']"
generic_metrics_rn_1,Generic Metrics rn_1,"['num_images_total_ground_truth', 'num_images_total_predicted', 'num_images_recent_ground_truth', 'num_images_recent_predicted']"
segmentation_metrics_rn_1,Segmentation Metrics rn_1,"['precision', 'precision_positive', 'precision_negative', 'recall', 'recall_positive', 'recall_negative', 'f1', 'f1_positive', 'f1_negative', 'Dice', 'Dice_positive', 'Dice_negative', 'IOU', 'IOU_positive', 'IOU_negative', 'PixAcc', 'PixAcc_positive', 'PixAcc_negative']"
segmentation_metrics_rn,segmentation_metrics_rn,"['precision', 'precision_positive', 'precision_negative', 'recall', 'recall_positive', 'recall_negative', 'f1', 'f1_positive', 'f1_negative', 'Dice', 'Dice_positive', 'Dice_negative', 'IOU', 'IOU_positive', 'IOU_negative', 'PixAcc', 'PixAcc_positive', 'PixAcc_negative']"
generic_metrics_rn,generic_metrics_rn,"['num_images_total_ground_truth', 'num_images_total_predicted', 'num_images_recent_ground_truth', 'num_images_recent_predicted']"
generic_metrics_n2,Generic Metrics n2,"['num_images_total_ground_truth', 'num_images_total_predicted', 'num_images_recent_ground_truth', 'num_images_recent_predicted']"
generic_metrics_n1,Generic Metrics n1,"['num_images_total_ground_truth', 'num_images_total_predicted', 'num_images_recent_ground_truth', 'num_images_recent_predicted']"
segmentation_metrics_n1,Segmentation Metrics n1,"['precision', 'precision_positive', 'precision_negative', 'recall', 'recall_positive', 'recall_negative', 'f1', 'f1_positive', 'f1_negative', 'Dice', 'Dice_positive', 'Dice_negative', 'IOU', 'IOU_positive', 'IOU_negative', 'PixAcc', 'PixAcc_positive', 'PixAcc_negative']"
test_sample_model_performance,Test Sample Model Performance,"['sensitivity', 'specificity', 'gender_less40_fav_prediction_ratio']"
reirradiation_batch_api_performance,ReIrradiation Batch API Performance,"['setup_time', 'process_time']"


Note: First 10 records were displayed.


In [9]:
wos_util.get_monitor_definition(wos_client,monitor_name=openscale_monitor_name)

{'metadata': {'id': 'dummy_monitor_example_dima',
  'crn': 'crn:v1:bluemix:public:aiopenscale:us-south:a/na:00000000-0000-0000-0000-000000000000:monitor_definition:dummy_monitor_example_dima',
  'url': '/v2/monitor_definitions/dummy_monitor_example_dima',
  'created_at': '2023-09-08T14:28:05.310000Z',
  'created_by': 'kharlad'},
 'entity': {'name': 'dummy monitor example dima',
  'metrics': [{'name': 'sensitivity',
    'thresholds': [{'type': 'lower_limit', 'default': 0.8}],
    'expected_direction': 'increasing',
    'id': 'sensitivity'},
   {'name': 'specificity',
    'thresholds': [{'type': 'lower_limit', 'default': 0.9}],
    'expected_direction': 'increasing',
    'id': 'specificity'}],
  'tags': []}}